In [1]:
import numpy as np
import pickle
from pathlib import Path
import pandas as pd

In [2]:
calval_dir = Path.cwd()/'calval'
calval_location = 'central_valley'
# calval_location = 'texas'
# calval_location = 'oklahoma'
# calval_location = 'purtorico'
work_dir = calval_dir/calval_location

In [3]:
with open(work_dir/'approach2.pkl','rb') as f:
    dist,rel_measure, ifgs_date = pickle.load(f)

In [4]:
n_ifgs = len(dist)

In [5]:
n_bins = 100
bins = np.linspace(0.1,50.0,num=n_bins+1)

In [6]:
n_all = np.empty([n_ifgs,n_bins+1],dtype=int) # number of points for each ifgs and bins
n_pass = np.empty([n_ifgs,n_bins+1],dtype=int) # number of points pass
#ratio = np.empty([n_ifgs,n_bins+1]) # ratio
# the final column is the ratio as a whole
for i in range(n_ifgs):
    inds = np.digitize(dist[i],bins)
    for j in range(1,n_bins+1):
        rqmt = 3*(1+np.sqrt(dist[i][inds==j]))# mission requirement for i-th ifgs and j-th bins
        rem = rel_measure[i][inds==j] # relative measurement
        assert len(rqmt) == len(rem)
        n_all[i,j-1] = len(rem)
        n_pass[i,j-1] = np.count_nonzero(rem<rqmt)
    n_all[i,-1] = np.sum(n_all[i,0:-2])
    n_pass[i,-1] = np.sum(n_pass[i,0:-2])

In [7]:
def to_str(x:bool):
    if x==True:
        return 'true '
    elif x==False:
        return 'false '

In [8]:
ratio = n_pass/n_all
mean_ratio = np.array([np.mean(ratio[:,:-1],axis=1)])
ratio = np.hstack((ratio,mean_ratio.T))
thresthod = 0.683
#The assumed nature of Gaussian distribution gives a probability of 0.683 of being within one standard deviation.
success_or_fail = ratio>thresthod
success_or_fail_str = [list(map(to_str, x)) for x in success_or_fail]

In [9]:
columns = []
for i in range(n_bins):
    columns.append(f'{bins[i]:.2f}-{bins[i+1]:.2f}')
columns.append('total')

In [10]:
index = []
for i in range(len(ifgs_date)):
    index.append(ifgs_date[i,0].strftime('%Y%m%d')+'-'+ifgs_date[i,1].strftime('%Y%m%d'))

In [11]:
n_all_pd = pd.DataFrame(n_all,columns=columns,index=index)
n_pass_pd = pd.DataFrame(n_pass,columns=columns,index=index)
ratio_pd = pd.DataFrame(ratio,columns=columns+['mean'],index=index)
success_or_fail_pd = pd.DataFrame(success_or_fail_str,columns=columns+['mean'],index=index)

Number of data points in each bin:

In [12]:
n_all_pd

,0.10-0.60,0.60-1.10,1.10-1.60,1.60-2.10,2.10-2.60,2.60-3.09,3.09-3.59,3.59-4.09,4.09-4.59,4.59-5.09,...,45.51-46.01,46.01-46.51,46.51-47.01,47.01-47.51,47.51-48.00,48.00-48.50,48.50-49.00,49.00-49.50,49.50-50.00,total
20170108-20170120,31,60,74,130,147,189,208,267,285,291,...,1855,1823,1819,1871,1788,1823,1905,1857,1943,114461
20170114-20170126,15,49,104,133,128,193,218,223,265,297,...,1859,1759,1786,1853,1901,1817,1875,1865,1865,114636
20170120-20170201,23,48,90,121,142,162,221,251,280,353,...,1792,1812,1862,1806,1853,1780,1879,1846,1901,114296
20170225-20170309,20,47,83,112,144,176,199,219,271,266,...,1796,1838,1831,1894,1834,1877,1921,1820,1878,114489
20170321-20170402,23,50,87,107,152,188,208,256,277,300,...,1842,1868,1794,1797,1943,1860,1871,1843,1883,114774
20170414-20170426,25,47,78,117,159,198,210,268,266,305,...,1789,1825,1916,1805,1897,1774,1894,1913,1888,114919
20170508-20170520,26,64,68,108,138,182,213,229,264,309,...,1829,1852,1830,1767,1905,1862,1897,1866,1886,114705
20170601-20170613,28,53,88,107,165,174,213,225,282,292,...,1867,1789,1758,1814,1950,1884,1842,1856,1893,114487
20170625-20170707,28,47,96,124,146,170,217,253,283,302,...,1823,1805,1736,1861,1803,1896,1857,1923,1883,115207
20170719-20170731,17,54,82,120,152,154,206,246,282,287,...,1846,1755,1828,1771,1886,1814,1885,1832,1889,114151


Number of data points that below the curve:

In [13]:
n_pass_pd

,0.10-0.60,0.60-1.10,1.10-1.60,1.60-2.10,2.10-2.60,2.60-3.09,3.09-3.59,3.59-4.09,4.09-4.59,4.59-5.09,...,45.51-46.01,46.01-46.51,46.51-47.01,47.01-47.51,47.51-48.00,48.00-48.50,48.50-49.00,49.00-49.50,49.50-50.00,total
20170108-20170120,27,53,67,108,126,155,164,216,222,236,...,1625,1618,1602,1659,1572,1627,1687,1619,1731,99126
20170114-20170126,12,45,88,117,111,175,193,188,236,262,...,1708,1625,1658,1711,1761,1684,1750,1734,1714,104352
20170120-20170201,19,40,72,104,118,140,180,213,233,299,...,1483,1511,1552,1510,1539,1475,1522,1521,1570,94645
20170225-20170309,18,45,77,103,130,168,188,199,242,241,...,1582,1627,1586,1647,1612,1620,1686,1559,1621,101473
20170321-20170402,21,43,76,94,111,155,171,192,208,223,...,925,992,917,917,952,958,946,949,947,63874
20170414-20170426,24,44,76,112,151,192,198,257,253,292,...,1170,1169,1260,1152,1210,1103,1184,1181,1125,89730
20170508-20170520,24,58,64,102,127,170,200,211,250,276,...,1700,1718,1702,1639,1766,1729,1740,1732,1755,106402
20170601-20170613,27,47,82,98,156,157,202,213,260,266,...,1632,1596,1529,1579,1704,1638,1602,1641,1655,102043
20170625-20170707,28,46,90,117,136,156,201,233,251,259,...,1336,1341,1280,1376,1396,1430,1395,1439,1436,89062
20170719-20170731,14,51,80,114,144,147,199,236,264,268,...,1331,1248,1290,1267,1340,1306,1350,1298,1361,88397


Ratio of pass:

In [14]:
s = ratio_pd.style
s.set_table_styles([  # create internal CSS classes
    {'selector': '.true', 'props': 'background-color: #e6ffe6;'},
    {'selector': '.false', 'props': 'background-color: #ffe6e6;'},
], overwrite=False)
s.set_td_classes(success_or_fail_pd)

,0.10-0.60,0.60-1.10,1.10-1.60,1.60-2.10,2.10-2.60,2.60-3.09,3.09-3.59,3.59-4.09,4.09-4.59,4.59-5.09,5.09-5.59,5.59-6.09,6.09-6.59,6.59-7.09,7.09-7.58,7.58-8.08,8.08-8.58,8.58-9.08,9.08-9.58,9.58-10.08,10.08-10.58,10.58-11.08,11.08-11.58,11.58-12.08,12.08-12.57,12.57-13.07,13.07-13.57,13.57-14.07,14.07-14.57,14.57-15.07,15.07-15.57,15.57-16.07,16.07-16.57,16.57-17.07,17.07-17.57,17.57-18.06,18.06-18.56,18.56-19.06,19.06-19.56,19.56-20.06,20.06-20.56,20.56-21.06,21.06-21.56,21.56-22.06,22.06-22.55,22.55-23.05,23.05-23.55,23.55-24.05,24.05-24.55,24.55-25.05,25.05-25.55,25.55-26.05,26.05-26.55,26.55-27.05,27.05-27.55,27.55-28.04,28.04-28.54,28.54-29.04,29.04-29.54,29.54-30.04,30.04-30.54,30.54-31.04,31.04-31.54,31.54-32.04,32.04-32.54,32.54-33.03,33.03-33.53,33.53-34.03,34.03-34.53,34.53-35.03,35.03-35.53,35.53-36.03,36.03-36.53,36.53-37.03,37.03-37.52,37.52-38.02,38.02-38.52,38.52-39.02,39.02-39.52,39.52-40.02,40.02-40.52,40.52-41.02,41.02-41.52,41.52-42.02,42.02-42.52,42.52-43.01,43.01-43.51,43.51-44.01,44.01-44.51,44.51-45.01,45.01-45.51,45.51-46.01,46.01-46.51,46.51-47.01,47.01-47.51,47.51-48.00,48.00-48.50,48.50-49.00,49.00-49.50,49.50-50.00,total,mean
20170108-20170120,0.870968,0.883333,0.905405,0.830769,0.857143,0.820106,0.788462,0.808989,0.778947,0.810997,0.791541,0.810959,0.761773,0.814059,0.828508,0.816701,0.812741,0.812144,0.816822,0.827350,0.810855,0.817629,0.807044,0.828452,0.830269,0.819293,0.857923,0.830709,0.846793,0.825455,0.847363,0.847554,0.834475,0.826321,0.880536,0.858025,0.853635,0.854917,0.869522,0.841066,0.849644,0.831075,0.867216,0.862676,0.846709,0.859951,0.845840,0.867412,0.861961,0.843260,0.850000,0.862336,0.862566,0.872781,0.859275,0.856209,0.891273,0.871795,0.867107,0.862898,0.862491,0.879346,0.879679,0.881588,0.877646,0.874674,0.863968,0.879148,0.879156,0.871943,0.881463,0.868488,0.874924,0.858802,0.867390,0.877235,0.881829,0.878077,0.891682,0.874487,0.883965,0.880454,0.895097,0.880423,0.888060,0.884444,0.879740,0.887097,0.881864,0.882131,0.883811,0.876011,0.887548,0.880704,0.886692,0.879195,0.892485,0.885564,0.871836,0.890890,0.866024,0.856096
20170114-20170126,0.800000,0.918367,0.846154,0.879699,0.867188,0.906736,0.885321,0.843049,0.890566,0.882155,0.866477,0.845144,0.870732,0.861314,0.862700,0.861804,0.870155,0.902482,0.901582,0.890578,0.879282,0.875591,0.894895,0.878788,0.899593,0.889186,0.881491,0.875481,0.890428,0.887273,0.898864,0.900000,0.892673,0.890658,0.904137,0.906504,0.891176,0.913669,0.901304,0.899548,0.900909,0.893333,0.904632,0.902066,0.901993,0.910279,0.891081,0.900161,0.907482,0.917299,0.908869,0.907774,0.916914,0.907436,0.923423,0.915869,0.925220,0.918519,0.913357,0.913255,0.912219,0.908046,0.901882,0.928994,0.922087,0.907432,0.925470,0.900064,0.917148,0.911510,0.910533,0.920489,0.914067,0.917872,0.917008,0.915711,0.917189,0.930450,0.917690,0.924653,0.908516,0.917784,0.911019,0.909831,0.922897,0.914390,0.913972,0.935108,0.918163,0.906578,0.908938,0.918774,0.923820,0.928331,0.923368,0.926355,0.926802,0.933333,0.929759,0.919035,0.910290,0.901999
20170120-20170201,0.826087,0.833333,0.800000,0.859504,0.830986,0.864198,0.814480,0.848606,0.832143,0.847025,0.815710,0.818182,0.818898,0.780430,0.811765,0.805785,0.828740,0.826715,0.801394,0.813131,0.812500,0.790297,0.804615,0.827489,0.820717,0.812500,0.825700,0.828425,0.828916,0.826087,0.826542,0.842222,0.833149,0.822340,0.822811,0.806641,0.826903,0.834615,0.849151,0.837766,0.841270,0.831588,0.838944,0.814503,0.824138,0.831452,0.830998,0.820908,0.827036,0.830453,0.817620,0.834981,0.835878,0.816563,0.822909,0.825038,0.806787,0.837307,0.848341,0.848189,0.846154,0.827093,0.835739,0.827562,0.822570,0.835366,0.835052,0.838320,0.834424,0.829721,0.827497,0.817375,0.819585,0.814698,0.834341,0.824653,0.819488,0.831953,0.830660,0.841032,0.837223,0.832850,0.832553,0.825243,0.819606,0.832772,0.814469,0.837134,0.832164,0.822126,0.836446,0.827567,0.833885,0.833512,0.836102,0.830545,0.828652,0.810005,0.823944,0.825881,0.828069,0.827074
20170225-20170309,0

Percentage of interferograms passes the requirement (0.683):

In [15]:
np.count_nonzero(ratio_pd['total']>thresthod)/n_ifgs

0.9183673469387755

In [16]:
np.count_nonzero(ratio_pd['mean']>thresthod)/n_ifgs

0.9591836734693877